In [3]:
import cv2
import numpy as np

def moravec_corner_detection(image, threshold):
    """
    Detect corners in an image using the Moravec corner detection algorithm.

    Args:
    - image: Input grayscale image.
    - threshold: Threshold for corner detection. Pixels with a difference greater than this threshold are considered corners.

    Returns:
    - output_img: Image with detected corners highlighted in green.
    """

    # Convert input image to RGB format for visualization
    output_img = cv2.cvtColor(image.copy(), cv2.COLOR_GRAY2RGB)

    # Define shifts for neighboring pixels
    xy_shifts = [(1, 0), (1, 1), (0, 1), (-1, 1)]

    # Iterate over each pixel in the image
    for y in range(1, image.shape[0]-1):
        for x in range(1, image.shape[1]-1):
            # Initialize the variable E to a large value. It serves as a placeholder for the minimum difference
            # in intensity between the current pixel and its neighbors during the Moravec corner detection algorithm.
            E = 100000

            # Iterate over each neighboring pixel shift
            for shift in xy_shifts:
                # Compute the difference in intensity between the current pixel and its neighbor
                diff = int(image[y + shift[1], x + shift[0]]) - int(image[y, x])
                diff = diff * diff

                # Update E with the minimum difference found so far
                if diff < E:
                    E = diff
            
            # If the minimum difference (E) is above the threshold, mark the pixel as a corner
            if E > threshold:
                # Draw a green circle around the detected corner
                cv2.circle(output_img, (x, y), 5, (0, 255, 0), -1)
    
    return output_img

# Load input image
image = cv2.imread("D:/Muhammad/College/Computer Vision/Assignments/Assignment 3/Harris-Corner-600x570.jpg", cv2.IMREAD_GRAYSCALE)

# Set threshold for corner detection
threshold = 1000

# Perform Moravec corner detection
corners = moravec_corner_detection(image, threshold)

# Display the detected corners
cv2.imshow('Moravec Corner Detection', corners)
cv2.waitKey(0)
cv2.destroyAllWindows()
